In [ ]:
# 2. Facial recognition using OpenCV and deep learning for binary classification.

In [4]:
import os
import numpy as np
import cv2
from sklearn.preprocessing import LabelEncoder

In [5]:
# Function to detect face and extract it from an image
def extract_face(image_path, face_cascade):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    
    if len(faces) > 0:
        x, y, w, h = faces[0]
        return img[y:y+h, x:x+w]
    return None

In [6]:
# Function to prepare data: load images and extract faces
def prepare_data(data_directory, face_cascade):
    faces, labels = [], []
    for label_name in os.listdir(data_directory):
        label_dir = os.path.join(data_directory, label_name)
        if os.path.isdir(label_dir):
            for img_name in os.listdir(label_dir):
                img_path = os.path.join(label_dir, img_name)
                face = extract_face(img_path, face_cascade)
                if face is not None:
                    faces.append(cv2.cvtColor(face, cv2.COLOR_BGR2GRAY))
                    labels.append(label_name)
    return faces, labels


In [7]:
# Train a face recognizer with the dataset
def train_recognizer(data_directory, face_cascade):
    faces, labels = prepare_data(data_directory, face_cascade)
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(labels)
    
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.train(faces, np.array(encoded_labels))
    
    return recognizer, label_encoder


In [8]:
# Recognize the face from a new image
def recognize_face(image_path, recognizer, face_cascade, label_encoder):
    face = extract_face(image_path, face_cascade)
    if face is not None:
        gray_face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        label, confidence = recognizer.predict(gray_face)
        return label_encoder.inverse_transform([label])[0], confidence
    return None, None


In [ ]:
# Example of running the face recognition system
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
data_directory = r'path_to_your_dataset'  # Update with the correct dataset path


In [ ]:
# Train the recognizer
recognizer, label_encoder = train_recognizer(data_directory, face_cascade)


In [ ]:
# Example test image for face recognition
image_path = r'path_to_test_image/Sad_Person.jpeg'  # Update with a test image path
name, confidence = recognize_face(image_path, recognizer, face_cascade, label_encoder)
print(f"Predicted name: {name}, Confidence: {confidence}")
